# Affinitron: Text-to-Audio Spectral Analysis

This notebook analyzes text to build spectral signatures of words and synthesizes them as audio.

In [ ]:
# 1) Imports
import re, math
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import IPython.display as ipd
plt.rcParams['figure.figsize'] = (10,7)

In [ ]:
# simple tokenizer
def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]"," ", text)
    return [t for t in text.split() if t]

# sliding-window co-occurrence
def cooccurrence(tokens, window=3):
    co = defaultdict(Counter)
    for i,w in enumerate(tokens):
        left = max(0, i-window)
        right = min(len(tokens), i+window+1)
        for j in range(left, right):
            if j==i: continue
            co[w][tokens[j]] += 1
    return co

corpus_text = '''In recent years the field of artificial intelligence has experienced remarkable growth. Researchers build models that learn from large amounts of text. This text often contains historical, technical, and poetic passages. Understanding how words relate -- not only by frequency but by deeper affinities -- is essential. The Affinitron idea proposes a harmonic organization of knowledge, where grammar, history, and usage create a spectral signature.

Consider words like 'apple', 'fruit', 'eat', 'tree', 'garden', 'technology', 'computer', 'algorithm'. In some contexts 'apple' and 'fruit' are strongly related; in others 'apple' might be about the company or a device. Etymology, tempo of adoption, and syntactic roles help reveal the different layers of meaning.

We will use this medium-sized toy corpus which is synthetic but varied enough to illustrate structure.
'''

tokens = tokenize(corpus_text)
print('Tokens:', len(tokens))
print('Unique tokens:', len(set(tokens)))

co = cooccurrence(tokens, window=3)
for w,c in list(co.items())[:12]:
    print(w, '->', c.most_common(5))

In [ ]:
# 3) Partial estimation (simple heuristic)
rank_to_ratio = {1:1.0, 2:2.0, 3:1.5, 4:1.3333, 5:1.25, 6:1.2}

def topk_contexts(co, k=6):
    return {w: [x for x,_ in c.most_common(k)] for w,c in co.items()}

def estimate_partials(top_contexts, k=3):
    out = {}
    for w, ctxs in top_contexts.items():
        ratios = [rank_to_ratio.get(i+1, 1.0 + 0.05*(i+1)) for i in range(min(k, len(ctxs)))]
        amps = [ (k-i)/sum(range(1,k+1)) for i in range(len(ratios)) ]
        if not ratios:
            ratios = [1.0,2.0,1.5]
            amps = [0.6,0.3,0.1]
        out[w] = (ratios, amps)
    return out

top_ctx = topk_contexts(co, k=6)
partials = estimate_partials(top_ctx, k=3)
print('Example partials for some words:')
for w in list(partials.keys())[:8]:
    print(w, '->', partials[w])

In [ ]:
# 4) Tempo (alpha) estimation using simulated time-slices
N_slices = 5
slice_len = max(1, len(tokens)//N_slices)
slices = [tokens[i*slice_len:(i+1)*slice_len] for i in range(N_slices)]
from collections import Counter
freq_slices = [Counter(s) for s in slices]

def estimate_alpha(word, freq_slices):
    svals = np.array([freq_slices[i].get(word, 0) for i in range(len(freq_slices))])
    if svals.sum() == 0:
        return 1.0
    mean = svals.mean(); var = svals.var()
    burst = var/(mean+1e-6)
    recency_index = np.dot(svals, np.arange(1,len(freq_slices)+1)) / (svals.sum()+1e-6)
    alpha = 1.0 + min(3.0, burst*0.5) + (recency_index/(len(freq_slices)*2))
    return float(alpha)

print('Alpha examples:')
for w in ['apple','technology','text','affinitron','fruit','eat']:
    print(w, estimate_alpha(w, freq_slices))

In [ ]:
# 5) Build signatures for top-V words and compute affinity graph
from collections import Counter
word_counts = Counter(tokens)
VOCAB_SIZE = 80
vocab = [w for w,_ in word_counts.most_common(VOCAB_SIZE)]

def base_pitch(word):
    verbs = set(['is','are','build','learn','contain','use','consider','show','reveal','experience','has','have'])
    if word in verbs:
        return 220.0
    if len(word) <= 3:
        return 440.0
    return 110.0

signatures = {}
for w in vocab:
    ratios, amps = partials.get(w, ([1.0,2.0,1.5],[0.6,0.3,0.1]))
    freqs = [base_pitch(w)*r for r in ratios]
    alpha = estimate_alpha(w, freq_slices)
    signatures[w] = {'freqs': freqs, 'amps': amps, 'alpha': alpha}

# affinity functions
beta_f = 200.0; beta_alpha = 2.0; w_f=0.7; w_a=0.3

def harmonic_overlap(su, sv):
    fu = np.array(su['freqs']); fv = np.array(sv['freqs'])
    au = np.array(su['amps']); av = np.array(sv['amps'])
    sim=0.0; wt=0.0
    for i in range(len(fu)):
        for j in range(len(fv)):
            delta = abs(fu[i]-fv[j]) / max(fu[i], fv[j])
            s = math.exp(-beta_f*(delta**2))
            w = au[i]*av[j]
            sim += w*s; wt += w
    return sim/wt if wt>0 else 0.0

def tempo_compat(su, sv):
    d = math.log(su['alpha']+1e-8) - math.log(sv['alpha']+1e-8)
    return math.exp(-beta_alpha*(d**2))

V=len(vocab)
A = np.zeros((V,V))
for i,u in enumerate(vocab):
    for j,v in enumerate(vocab):
        if i==j: continue
        A[i,j] = w_f*harmonic_overlap(signatures[u], signatures[v]) + w_a*tempo_compat(signatures[u], signatures[v])

# build graph
G = nx.Graph()
for w in vocab:
    G.add_node(w)
threshold = 0.15
for i in range(V):
    for j in range(i+1,V):
        if A[i,j] > threshold:
            G.add_edge(vocab[i], vocab[j], weight=float(A[i,j]))
print('Graph nodes:', G.number_of_nodes(), 'edges:', G.number_of_edges())

In [ ]:
# 6) Visualize graph
pos = nx.spring_layout(G, seed=42)
plt.figure(figsize=(10,8))
weights = [G[u][v]['weight']*6 for u,v in G.edges()]
nx.draw_networkx_nodes(G, pos, node_size=200, node_color='lightblue')
nx.draw_networkx_edges(G, pos, width=weights, alpha=0.7)
nx.draw_networkx_labels(G, pos, font_size=8)
plt.title('Affinitron affinity graph (toy corpus)')
plt.axis('off')
plt.show()

# Example path
def affinity_path(start, L=6):
    path=[start]
    cur=start
    for _ in range(L-1):
        nbrs = sorted(G[cur].items(), key=lambda kv: kv[1]['weight'], reverse=True)
        if not nbrs: break
        cur=nbrs[0][0]
        path.append(cur)
    return path

print('Path from apple:', affinity_path('apple'))
print('Path from technology:', affinity_path('technology'))

## Audio Synthesis

Here we add the ability to synthesize the spectral signatures as audio.

In [ ]:
# 7) Audio Synthesis Functions

def generate_tone(freqs, amps, duration=1.0, rate=44100):
    t = np.linspace(0, duration, int(rate * duration), endpoint=False)
    wave = np.zeros_like(t)
    
    # Normalize amps
    amps = np.array(amps)
    if amps.sum() > 0:
        amps = amps / amps.sum()
    
    for f, a in zip(freqs, amps):
        wave += a * np.sin(2 * np.pi * f * t)
    
    return wave, rate

def apply_envelope(wave, alpha, rate=44100):
    # Simple ADSR-like envelope based on alpha (tempo)
    # Higher alpha -> faster attack/decay (more percussive)
    # Lower alpha -> slower attack/decay (more pad-like)
    
    duration = len(wave) / rate
    t = np.linspace(0, duration, len(wave), endpoint=False)
    
    # Map alpha to envelope parameters
    # alpha ranges roughly 1.0 to 4.0+
    attack_time = 0.05 / max(1.0, alpha * 0.5)
    decay_time = 0.2 / max(1.0, alpha * 0.5)
    
    envelope = np.ones_like(wave)
    
    # Attack
    attack_samples = int(attack_time * rate)
    if attack_samples > 0:
        envelope[:attack_samples] = np.linspace(0, 1, attack_samples)
        
    # Decay/Release (simplified)
    decay_samples = int(decay_time * rate)
    if decay_samples > 0 and len(wave) > attack_samples:
        remaining = len(wave) - attack_samples
        # Exponential decay
        decay_curve = np.exp(-np.linspace(0, 5, remaining))
        envelope[attack_samples:] = decay_curve
        
    return wave * envelope

def play_signature(word, duration=1.0):
    if word not in signatures:
        print(f"Word '{word}' not in signatures.")
        return
        
    sig = signatures[word]
    print(f"Playing '{word}': Freqs={np.round(sig['freqs'], 1)}, Alpha={sig['alpha']:.2f}")
    
    wave, rate = generate_tone(sig['freqs'], sig['amps'], duration=duration)
    wave = apply_envelope(wave, sig['alpha'], rate)
    
    # Normalize to prevent clipping
    if np.max(np.abs(wave)) > 0:
        wave = wave / np.max(np.abs(wave)) * 0.9
        
    return ipd.Audio(wave, rate=rate)

In [ ]:
# 8) Interactive Audio Demo

# Play a few words
display(play_signature('apple'))
display(play_signature('technology'))
display(play_signature('is'))

# Play a path
print("\nPlaying path from 'apple':")
path = affinity_path('apple')
for w in path:
    display(play_signature(w, duration=0.8))